# Data preparing

In [1]:
import numpy as np
import jieba
import tensorflow as tf

In [2]:
# word embedding look up loading
word2vec_lookup = np.load('word2vec_gensim.npy')
word2vec_lookup = word2vec_lookup[None][0]

In [3]:
# one hot encoding
words2oneHot = dict()
words = np.array(list(word2vec_lookup.keys()))
voca_size = len(words)
for i in range(voca_size):
    words2oneHot[words[i]] = np.zeros(voca_size).astype('int')
    words2oneHot[words[i]][i] = 1
oneHotPos2vec_lookup = dict()
oneHotPos2words = dict()
for key, val in words2oneHot.items():
    oneHotPos2vec_lookup[np.argmax(words2oneHot[key])] = word2vec_lookup[key]
    oneHotPos2words[np.argmax(words2oneHot[key])] = key

In [4]:
vec_list = [list(oneHotPos2vec_lookup[i]) for i in range(len(oneHotPos2vec_lookup.keys()))]

In [5]:
# romance condor heroes vectors
sent_len = 10

with open('romance_condor_heroes.txt', 'r') as f:
    data = list(f)
sentences = list()
for i in range(len(data)):
    sentences += data[i].replace(' ', '').replace('“', '').replace('，', '').replace('”', '')\
                 .replace('、', '').replace('！', '').replace('‘', '').replace('’', '').replace('：', '')\
                 .replace('？', '').replace('…', '').split('。')
print('Pre-processing done.')
sent_seg = list()
jieba.set_dictionary('dict.txt.big')
sent_seg_too_long = list()
f_s = list()
for i in range(len(sentences)):
    s = sentences[i]
    s = list(jieba.cut(s))
    if len(s) > sent_len:
        f = int(len(s)/sent_len)
        f_ = sent_len - len(s) + int(len(s)/sent_len)*sent_len
        for j in range(f):
            sent_seg.append(s[j*sent_len:j*sent_len+sent_len])
            f_s.append(0)
        unknowns = ['<unknown>' for k in range(f_)]
        if len(unknowns) >= sent_len/2:
            continue
        sent_seg.append(s[j*sent_len+sent_len:]+unknowns)
        sent_seg_too_long.append(s)
    else:
        f_ = 0
        unknowns = ['<unknown>' for j in range(sent_len-len(s))]
        s += unknowns
        sent_seg.append(s)
    f_s.append(f_)
sent_seg = np.array(sent_seg)
print('word segmentation done.', 'len =', len(sent_seg), len(sent_seg_too_long))
sent_seg.shape

Building prefix dict from /media/warrents/6A085EA7085E7255/GoogleDriveSync/WarrenTS/Code/Code/Python2.7/IPython_Notebooks/MachineLearning/pecu_course/Text_mining_LSTM/dict.txt.big ...
Loading model from cache /tmp/jieba.u86847c3b15d467aa53d8311fcf6be389.cache


Pre-processing done.


Loading model cost 1.207 seconds.
Prefix dict has been built succesfully.


word segmentation done. len = 46726 6055


(46726, 10)

In [6]:
__ = dict()
for i in f_s:
    if i in __:
        __[i] += 1
    else:
        __[i] = 1
__

{0: 40671, 1: 1385, 2: 1500, 3: 1560, 4: 1610}

 - Batch data function

In [7]:
def batch_X(i, batch):
    batch_x = list()
    for s in sent_seg[i:i+batch]:
        v = list()
        for w in s:
            if w is '<unknown>':
                v.append(vec_list[np.random.randint(len(vec_list))])
            elif w in word2vec_lookup:
                v.append(word2vec_lookup[w])
            else:
                v.append(vec_list[np.random.randint(len(vec_list))])
        batch_x.append(v)
    batch_x = np.array(batch_x)
    return batch_x
def batch_Y(i, batch):
    batch_y = list()
    for s in sent_seg[i+1:i+batch+1]:
        v = list()
        for w in s:
            if w is '<unknown>':
                v.append(np.random.randint(len(vec_list)))
            elif w in words2oneHot:
                v.append(np.argmax(words2oneHot[w]))
            else:
                v.append(np.random.randint(len(vec_list)))
        batch_y.append(v)
    batch_y = np.array(batch_y)
    return batch_y

In [8]:
batch_x = batch_X(0, 50)
batch_x2 = batch_X(1, 50)
batch_y = batch_Y(0, 50)
batch_x.shape, batch_x[45], batch_x[1]==batch_x2[0] # (batch size, sent len, vec len)

((50, 10, 1024),
 array([[-0.03225205,  0.02530756, -0.0233059 , ..., -0.21001685,
          0.1163154 , -0.11730645],
        [-0.08833244,  0.16655062, -0.02170643, ..., -0.3837541 ,
          0.16896005, -0.23355156],
        [ 0.11398   ,  0.09539733,  0.16519009, ..., -0.37131354,
          0.01709613, -0.18429933],
        ..., 
        [ 0.09077873,  0.00354263,  0.0945929 , ..., -0.0750194 ,
          0.08481972, -0.08937261],
        [ 0.05024337,  0.05750255,  0.10373137, ..., -0.10610861,
          0.0774331 , -0.11111267],
        [ 0.05258827, -0.03282837,  0.05547719, ..., -0.00375922,
          0.02663728, -0.01097239]], dtype=float32),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True],
        ..., 
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,

# LSTM

In [9]:
import tensorflow as tf

 - Hyperparameters

In [10]:
vec_size = len(word2vec_lookup['<unknown>'])
oneHot_size = len(words2oneHot['<unknown>'])
enc_len = sent_len
dec_len = enc_len
n_layer1 = 1024
l_r = 1e-3
epoch = 500
batch_size = 128

 - Weight and bias functions

In [11]:
# generate random weight and bias
def _weight(shape):
    initial = tf.truncated_normal(shape, stddev=1.)
    return tf.Variable(initial)

def _bias(shape):
    initial = tf.constant(1., shape=shape)
    return tf.Variable(initial)

 - Encoder and Decoder input
 <br> 15:15

In [12]:
# word vectors constant
vecs = tf.constant(vec_list)

# input - word vectors
word_vec = [tf.placeholder(tf.float32, shape=[None, vec_size], name='word_vec') for i in range(enc_len)]    

# encoder input
w_enc = _weight([vec_size, n_layer1])
b_enc = _bias([n_layer1])
enc_inp = [tf.nn.dropout(tf.sigmoid(tf.matmul(word_vec[i], w_enc)+b_enc), keep_prob=.8, name='enc_inp') \
           for i in range(enc_len)]

# training target - word one-hot
word_target = [tf.placeholder(tf.int32, shape=[None], name='word_target') for i in range(dec_len)]

# one-hot to word vectors
target = [tf.nn.embedding_lookup(vecs, word_target[i]) for i in range(dec_len-1)]
target = [tf.zeros_like(target[0], dtype=np.float32, name="GO")] + target

# decoder input
w_dec = _weight([vec_size, n_layer1])
b_dec = _bias([n_layer1])
dec_inp = [tf.nn.dropout(tf.nn.relu(tf.matmul(target[i], w_dec)+b_dec), keep_prob=.8, name='dec_inp') \
           for i in range(dec_len)]

 - Seq2seq model

In [13]:
n_hiddens = 3
cells = [tf.contrib.rnn.BasicLSTMCell(n_layer1, forget_bias=1.0) for i in range(n_hiddens)]
cell = tf.contrib.rnn.MultiRNNCell(cells)

dec_outputs, dec_memory = tf.contrib.legacy_seq2seq.basic_rnn_seq2seq(
        enc_inp, 
        dec_inp, 
        cell
    )

w_dec_o = _weight([n_layer1, oneHot_size])
b_dec_o = _weight([oneHot_size])
dec_outputs = [tf.matmul(dec_outputs[i], w_dec_o)+b_dec_o for i in range(dec_len)]

 - Loss function and optimizer

In [16]:
loss = list()
loss_all = 0
accu = list()
for y_hat, y_real in zip(dec_outputs, word_target):
    #loss.append(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y_real)))
    print(y_hat, y_real)
    #accu += tf.reduce_mean(tf.cast(tf.equal(y_hat, y_real), tf.float32))
    loss_all += tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_hat, labels=y_real)
loss_all = tf.reduce_mean(loss_all/dec_len)
lr_decay = 0.92  # default: 0.9 . Simulated annealing.
momentum = 0.5  # default: 0.0 . Momentum technique in weights update
op = tf.train.RMSPropOptimizer(l_r, decay=lr_decay, momentum=momentum).minimize(loss_all)
#ops = [tf.train.AdamOptimizer(l_r).minimize(l) for l in loss]
#op3 = tf.train.AdagradOptimizer(l_r).minimize(loss)

Tensor("add_20:0", shape=(?, 12258), dtype=float32) Tensor("word_target:0", shape=(?,), dtype=int32)
Tensor("add_21:0", shape=(?, 12258), dtype=float32) Tensor("word_target_1:0", shape=(?,), dtype=int32)
Tensor("add_22:0", shape=(?, 12258), dtype=float32) Tensor("word_target_2:0", shape=(?,), dtype=int32)
Tensor("add_23:0", shape=(?, 12258), dtype=float32) Tensor("word_target_3:0", shape=(?,), dtype=int32)
Tensor("add_24:0", shape=(?, 12258), dtype=float32) Tensor("word_target_4:0", shape=(?,), dtype=int32)
Tensor("add_25:0", shape=(?, 12258), dtype=float32) Tensor("word_target_5:0", shape=(?,), dtype=int32)
Tensor("add_26:0", shape=(?, 12258), dtype=float32) Tensor("word_target_6:0", shape=(?,), dtype=int32)
Tensor("add_27:0", shape=(?, 12258), dtype=float32) Tensor("word_target_7:0", shape=(?,), dtype=int32)
Tensor("add_28:0", shape=(?, 12258), dtype=float32) Tensor("word_target_8:0", shape=(?,), dtype=int32)
Tensor("add_29:0", shape=(?, 12258), dtype=float32) Tensor("word_target_9:0

In [17]:
op#, ops

<tf.Operation 'RMSProp_1' type=NoOp>

### - ***Training***

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [19]:
import time
for epo in range(epoch):
    print('Epoch', epo+1)
    t_start = time.time()
    for i in range(int(len(sent_seg)/batch_size)):
        batch_x = batch_X(i*batch_size, batch_size)
        batch_y = batch_Y(i*batch_size, batch_size)
        feed_dict = {word_vec[t]: batch_x[:, t] for t in range(enc_len)}
        feed_dict.update({word_target[t]: batch_y[:, t] for t in range(dec_len)})
        
        _ = sess.run([op], feed_dict=feed_dict)
        #lo, ac = sess.run([loss, accu], feed_dict=feed_dict)
        if (i+1)%50 == 0:
            lo = sess.run([loss_all], feed_dict=feed_dict)
            print('Epoch:', epo+1, 
                  ', iter:', i+1, 
                  ', loss:', lo)
        #if (epo+1)%10 == 0:
    #-----test-----
    batch_x = batch_X(1886, 1)
    batch_y = batch_Y(1886, 1)
    feed_dict = {word_vec[t]: batch_x[:, t] for t in range(enc_len)}
    feed_dict.update({word_target[t]: batch_y[:, t] for t in range(dec_len)})
    test = sess.run([dec_outputs], feed_dict=feed_dict)
    test = test[0]
    w_ = list()
    for i in range(sent_len):
        w_.append(test[i][0])
    print('Input: '+''.join(sent_seg[1886]))
    print('Output: '+''.join([oneHotPos2words[np.argmax(w)] for w in w_ ]))
    t_end = time.time()
    print('--------------------------Time span:', t_end-t_start, '--------------------------')

Epoch 1
Epoch: 1 , iter: 50 , loss: [9.1700363]
Epoch: 1 , iter: 100 , loss: [25.51412]
Epoch: 1 , iter: 150 , loss: [8.7245703]
Epoch: 1 , iter: 200 , loss: [8.5413151]
Epoch: 1 , iter: 250 , loss: [8.2176428]
Epoch: 1 , iter: 300 , loss: [7.6062489]
Epoch: 1 , iter: 350 , loss: [7.8964453]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 尹克西尹克西了的的的的的的的
--------------------------Time span: 95.8706316947937 --------------------------
Epoch 2
Epoch: 2 , iter: 50 , loss: [7.2189388]
Epoch: 2 , iter: 100 , loss: [7.0907927]
Epoch: 2 , iter: 150 , loss: [7.3824849]
Epoch: 2 , iter: 200 , loss: [7.3347039]
Epoch: 2 , iter: 250 , loss: [7.4593081]
Epoch: 2 , iter: 300 , loss: [7.2452312]
Epoch: 2 , iter: 350 , loss: [7.7257671]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 尹克西的了的的的的<unknown><unknown>的
--------------------------Time span: 95.17282438278198 --------------------------
Epoch 3
Epoch: 3 , iter: 50 , loss: [6.9821329]
Epoch: 3 , iter: 100 , loss: [6.9110227]
Epoch: 3 , iter: 150 , loss: [7.2125907]
Epoch: 3

Epoch: 19 , iter: 350 , loss: [4.5051937]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 尹克西往這在的是便在了在
--------------------------Time span: 94.90832257270813 --------------------------
Epoch 20
Epoch: 20 , iter: 50 , loss: [4.8275542]
Epoch: 20 , iter: 100 , loss: [4.5028934]
Epoch: 20 , iter: 150 , loss: [5.0572748]
Epoch: 20 , iter: 200 , loss: [4.8621454]
Epoch: 20 , iter: 250 , loss: [4.7001371]
Epoch: 20 , iter: 300 , loss: [4.7792435]
Epoch: 20 , iter: 350 , loss: [4.1061592]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 張君寶交給這拍上空張君寶了了<unknown>
--------------------------Time span: 94.9536542892456 --------------------------
Epoch 21
Epoch: 21 , iter: 50 , loss: [4.5421524]
Epoch: 21 , iter: 100 , loss: [4.1924763]
Epoch: 21 , iter: 150 , loss: [4.8036041]
Epoch: 21 , iter: 200 , loss: [4.6091862]
Epoch: 21 , iter: 250 , loss: [4.3649569]
Epoch: 21 , iter: 300 , loss: [4.3865471]
Epoch: 21 , iter: 350 , loss: [3.7979214]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 張君寶這一這拍抓住三便向尹克西尹克西
--------------------------Time sp

Epoch: 38 , iter: 250 , loss: [2.3696473]
Epoch: 38 , iter: 300 , loss: [1.785863]
Epoch: 38 , iter: 350 , loss: [0.42829326]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 只見個見到跟經書那頭漢子躥了張君寶
--------------------------Time span: 94.7793915271759 --------------------------
Epoch 39
Epoch: 39 , iter: 50 , loss: [2.1018345]
Epoch: 39 , iter: 100 , loss: [1.7100921]
Epoch: 39 , iter: 150 , loss: [2.2554348]
Epoch: 39 , iter: 200 , loss: [1.8275955]
Epoch: 39 , iter: 250 , loss: [2.2996116]
Epoch: 39 , iter: 300 , loss: [1.7805218]
Epoch: 39 , iter: 350 , loss: [0.39275753]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 倒地往他跟傳到巨闕可是已這張君寶
--------------------------Time span: 94.7673590183258 --------------------------
Epoch 40
Epoch: 40 , iter: 50 , loss: [1.9025908]
Epoch: 40 , iter: 100 , loss: [1.6983662]
Epoch: 40 , iter: 150 , loss: [2.1105382]
Epoch: 40 , iter: 200 , loss: [1.7514148]
Epoch: 40 , iter: 250 , loss: [2.1966312]
Epoch: 40 , iter: 300 , loss: [1.7939216]
Epoch: 40 , iter: 350 , loss: [0.22827978]
Inp

Epoch: 57 , iter: 150 , loss: [1.6905525]
Epoch: 57 , iter: 200 , loss: [1.4419811]
Epoch: 57 , iter: 250 , loss: [1.9485006]
Epoch: 57 , iter: 300 , loss: [1.4574952]
Epoch: 57 , iter: 350 , loss: [0.12286306]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 尹克西我漢出拳成安危嗎了紅印嗎
--------------------------Time span: 94.70639634132385 --------------------------
Epoch 58
Epoch: 58 , iter: 50 , loss: [1.6126339]
Epoch: 58 , iter: 100 , loss: [1.4710027]
Epoch: 58 , iter: 150 , loss: [1.7628765]
Epoch: 58 , iter: 200 , loss: [1.4740365]
Epoch: 58 , iter: 250 , loss: [1.9410123]
Epoch: 58 , iter: 300 , loss: [1.5176814]
Epoch: 58 , iter: 350 , loss: [0.11273313]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 黃藥師聽你尹克西大陣共分盡數將處斜那
--------------------------Time span: 94.73714828491211 --------------------------
Epoch 59
Epoch: 59 , iter: 50 , loss: [1.7009177]
Epoch: 59 , iter: 100 , loss: [1.6043055]
Epoch: 59 , iter: 150 , loss: [1.7007158]
Epoch: 59 , iter: 200 , loss: [1.4771016]
Epoch: 59 , iter: 250 , loss: [1.9405792]
E

Epoch: 76 , iter: 50 , loss: [1.4449811]
Epoch: 76 , iter: 100 , loss: [1.3484876]
Epoch: 76 , iter: 150 , loss: [1.6889381]
Epoch: 76 , iter: 200 , loss: [1.3786843]
Epoch: 76 , iter: 250 , loss: [1.8331072]
Epoch: 76 , iter: 300 , loss: [1.4148734]
Epoch: 76 , iter: 350 , loss: [0.1121366]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 長在之念岩石當真有緣千里<unknown>神雕<unknown>耀目
--------------------------Time span: 94.69034051895142 --------------------------
Epoch 77
Epoch: 77 , iter: 50 , loss: [1.5284402]
Epoch: 77 , iter: 100 , loss: [1.4159336]
Epoch: 77 , iter: 150 , loss: [1.5876354]
Epoch: 77 , iter: 200 , loss: [1.355979]
Epoch: 77 , iter: 250 , loss: [1.8463089]
Epoch: 77 , iter: 300 , loss: [1.3898413]
Epoch: 77 , iter: 350 , loss: [0.075091049]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 長把丈夫瀟湘子的搶<unknown>討上都
--------------------------Time span: 94.70970845222473 --------------------------
Epoch 78
Epoch: 78 , iter: 50 , loss: [1.4565415]
Epoch: 78 , iter: 100 , loss: [1.3932244]
Epoch: 78 , iter: 150 ,

...

Epoch: 422 , iter: 300 , loss: [1.5570636]
Epoch: 422 , iter: 350 , loss: [0.16964285]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 身子過去過來幾時的一起竟是不見<unknown><unknown>
--------------------------Time span: 94.56492161750793 --------------------------
Epoch 423
Epoch: 423 , iter: 50 , loss: [1.5412664]
Epoch: 423 , iter: 100 , loss: [1.5347759]
Epoch: 423 , iter: 150 , loss: [1.775938]
Epoch: 423 , iter: 200 , loss: [1.5109282]
Epoch: 423 , iter: 250 , loss: [1.9663618]
Epoch: 423 , iter: 300 , loss: [1.5760725]
Epoch: 423 , iter: 350 , loss: [0.29634431]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 大把這位絕情谷的天竺便史孟捷<unknown>身子
--------------------------Time span: 94.52137231826782 --------------------------
Epoch 424
Epoch: 424 , iter: 50 , loss: [1.5721956]
Epoch: 424 , iter: 100 , loss: [1.5241716]
Epoch: 424 , iter: 150 , loss: [1.7335274]
Epoch: 424 , iter: 200 , loss: [1.5187501]
Epoch: 424 , iter: 250 , loss: [1.9947019]
Epoch: 424 , iter: 300 , loss: [1.5100498]
Epoch: 424 , iter: 350 , loss: [0.15906468]


Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 出來手中一只搜尋前下都<unknown><unknown>通紅
--------------------------Time span: 94.5708749294281 --------------------------
Epoch 441
Epoch: 441 , iter: 50 , loss: [1.6662084]
Epoch: 441 , iter: 100 , loss: [1.5449791]
Epoch: 441 , iter: 150 , loss: [1.6653602]
Epoch: 441 , iter: 200 , loss: [1.6276586]
Epoch: 441 , iter: 250 , loss: [2.0982237]
Epoch: 441 , iter: 300 , loss: [1.4975441]
Epoch: 441 , iter: 350 , loss: [0.16032593]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 將手中我無幸周伯通聲勢了<unknown><unknown><unknown>
--------------------------Time span: 94.56435918807983 --------------------------
Epoch 442
Epoch: 442 , iter: 50 , loss: [1.5666039]
Epoch: 442 , iter: 100 , loss: [1.5535692]
Epoch: 442 , iter: 150 , loss: [1.6628565]
Epoch: 442 , iter: 200 , loss: [1.4939169]
Epoch: 442 , iter: 250 , loss: [1.9963007]
Epoch: 442 , iter: 300 , loss: [1.5116158]
Epoch: 442 , iter: 350 , loss: [0.21730557]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 將過去之間置之度外的不禁<unknown>出臉上<unknown>
-------

Epoch: 459 , iter: 50 , loss: [1.5623231]
Epoch: 459 , iter: 100 , loss: [1.5249422]
Epoch: 459 , iter: 150 , loss: [1.7921857]
Epoch: 459 , iter: 200 , loss: [1.5437407]
Epoch: 459 , iter: 250 , loss: [2.0480418]
Epoch: 459 , iter: 300 , loss: [1.557705]
Epoch: 459 , iter: 350 , loss: [0.20059818]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 將極適才一燈事箭卻招招起起
--------------------------Time span: 94.54829478263855 --------------------------
Epoch 460
Epoch: 460 , iter: 50 , loss: [1.5348285]
Epoch: 460 , iter: 100 , loss: [1.5516319]
Epoch: 460 , iter: 150 , loss: [1.6681949]
Epoch: 460 , iter: 200 , loss: [1.5183738]
Epoch: 460 , iter: 250 , loss: [2.0398483]
Epoch: 460 , iter: 300 , loss: [1.6134615]
Epoch: 460 , iter: 350 , loss: [0.19056275]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 從見全身一燈他後<unknown>了<unknown><unknown>
--------------------------Time span: 94.54795050621033 --------------------------
Epoch 461
Epoch: 461 , iter: 50 , loss: [1.5977011]
Epoch: 461 , iter: 100 , loss: [1.5279328]
Epoch: 461 

Epoch: 477 , iter: 100 , loss: [1.5716995]
Epoch: 477 , iter: 150 , loss: [1.774148]
Epoch: 477 , iter: 200 , loss: [1.5718627]
Epoch: 477 , iter: 250 , loss: [2.0813322]
Epoch: 477 , iter: 300 , loss: [1.5211849]
Epoch: 477 , iter: 350 , loss: [0.13198146]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 怪見她他到《壓出<unknown>架
--------------------------Time span: 94.50030446052551 --------------------------
Epoch 478
Epoch: 478 , iter: 50 , loss: [1.6079808]
Epoch: 478 , iter: 100 , loss: [1.5654435]
Epoch: 478 , iter: 150 , loss: [1.709541]
Epoch: 478 , iter: 200 , loss: [1.633812]
Epoch: 478 , iter: 250 , loss: [2.0135195]
Epoch: 478 , iter: 300 , loss: [1.4497906]
Epoch: 478 , iter: 350 , loss: [0.194924]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 便過去立時人馬招架一<unknown>的但<unknown>
--------------------------Time span: 94.5354232788086 --------------------------
Epoch 479
Epoch: 479 , iter: 50 , loss: [1.5909457]
Epoch: 479 , iter: 100 , loss: [1.5007833]
Epoch: 479 , iter: 150 , loss: [1.8327326]
Epoch: 479 , ite

Epoch: 495 , iter: 150 , loss: [1.7727587]
Epoch: 495 , iter: 200 , loss: [1.5969045]
Epoch: 495 , iter: 250 , loss: [2.16219]
Epoch: 495 , iter: 300 , loss: [1.6033942]
Epoch: 495 , iter: 350 , loss: [0.1357049]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 腿拿快來的一般壓了<unknown><unknown>
--------------------------Time span: 94.56375741958618 --------------------------
Epoch 496
Epoch: 496 , iter: 50 , loss: [1.5763739]
Epoch: 496 , iter: 100 , loss: [1.5817552]
Epoch: 496 , iter: 150 , loss: [1.7532098]
Epoch: 496 , iter: 200 , loss: [1.5102179]
Epoch: 496 , iter: 250 , loss: [2.0668478]
Epoch: 496 , iter: 300 , loss: [1.5041646]
Epoch: 496 , iter: 350 , loss: [0.16910255]
Input: 但聽篤篤篤之聲越來越近歐陽鋒忽地坐起
Output: 腿接一個在中後<unknown>在<unknown><unknown>
--------------------------Time span: 94.56444954872131 --------------------------
Epoch 497
Epoch: 497 , iter: 50 , loss: [1.5957212]
Epoch: 497 , iter: 100 , loss: [1.5731341]
Epoch: 497 , iter: 150 , loss: [1.853363]
Epoch: 497 , iter: 200 , loss: [1.5950677]


In [ ]:
# iter 350 check

In [22]:
saver = tf.train.Saver()
saver.save(sess, './models/'+time.strftime('%Y.%m.%d_%H%M%S')+'novel')

'./models/2018.01.05_133722novel'

In [23]:
# name of the tensor objects
word_vec, word_target, op, loss_all, dec_outputs

([<tf.Tensor 'word_vec:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_1:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_2:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_3:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_4:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_5:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_6:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_7:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_8:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'word_vec_9:0' shape=(?, 1024) dtype=float32>],
 [<tf.Tensor 'word_target:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_1:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_2:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_3:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_4:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_5:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'word_target_6:0' shape=(?,) dtype=int32>,
  <tf.Tenso

In [123]:
# test
inp_test = ['楊過', '怒道', '郭靖', '你', '還', '我', '爹爹', '命', '來', '<unknown>']
# resp ==> ['郭靖', '道', '他', '是', '我', '的', '你', '是', '惡人', '的']
def next_sent(inp):
    batch_x = np.array([[word2vec_lookup[inp[i]] for i in range(len(inp))]])
    batch_y = np.array([[0 for i in range(10)]])
    resp = list()
    for k in range(dec_len):
        feed_dict = {word_vec[t]: batch_x[:, t] for t in range(enc_len)}
        feed_dict.update({word_target[t]: batch_y[:, t] for t in range(dec_len)})

        resp.append(sess.run([dec_outputs], feed_dict=feed_dict)[0][k])
        batch_y[0][k] = np.argmax(resp[k])
    return resp
resp = next_sent(inp_test)

In [124]:
[oneHotPos2words[np.argmax(w)] for w in resp ]

['郭靖', '道', '我', '義父', '已', '聽', '他', '說', '你', '你']

In [116]:
sents = list()
inp = inp_test
for i in range(10):
    resp = next_sent(inp)
    sents.append([oneHotPos2words[np.argmax(w)] for w in resp ])
    inp = sents[i]
for j in range(len(sents)):
    print(''.join(sents[j]))

郭靖道我一生幫焦急麼楊過道我
爹爹媽媽長大死殺人心想他于是行走大師
只聽楊過朗探子啊喲說道郭二也明明白白了
郭襄見的蹤跡相見那料到每人和他兩個
難道他二人自來聯手自己在嘉興宮學藝說道
你大我經書我弟弟你她記得在
何處睡覺好了你的叫做老頑童<unknown><unknown>
就幾位你是否再這四人我也逃走受
陸立鼎走上來微微一笑說道我乃她們蓉脫手
不見見他掌力汗水之聲通紅軍加緊直至


In [81]:
batch_y[0][0] = np.argmax(resp[0][0])
np.argmax(resp[0][0]), batch_y

(1829, array([[1829,    0,    0,    0,    0,    0,    0,    0,    0,    0]]))

In [76]:
oneHotPos2words[1829]

'郭靖'